In [1]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbverbose.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
# default_exp agents.dqn.core

In [4]:
# export
# Python native modules
import os
from collections import deque
# Third party libs
import torch
from torch.nn import *
from fastcore.all import *
from fastai.learner import *
from fastai.torch_basics import *
from fastai.torch_core import *
from fastai.callback.all import *
# Local modules
from fastrl.data.block import *
from fastrl.data.gym import *
from fastrl.agent import *
from fastrl.core import *
from fastrl.memory.experience_replay import *

# DQN Core
> Contains the minimum DQN API.

In [5]:
# export
class DQN(Module):
    def __init__(self,state_sz:int,action_sz:int,hidden=512):
        self.layers=Sequential(
            Linear(state_sz,hidden),
            ReLU(),
            Linear(hidden,action_sz),
        )
    def forward(self,x): return self.layers(x)

In [6]:
dqn=DQN(4,2)
dqn(torch.randn((10,4)))

tensor([[ 0.0680, -0.2667],
        [-0.2827,  0.0976],
        [-0.4774,  0.0262],
        [-0.4901, -0.0640],
        [-0.0437, -0.1288],
        [-0.4818,  0.1282],
        [ 0.1730, -0.3305],
        [ 0.2023, -0.4457],
        [-0.3259, -0.1627],
        [-0.5810,  0.2117]], grad_fn=<AddmmBackward>)

In [7]:
# export
class ArgMaxFeed(AgentCallback):
    def before_action(self): 
        raw_action=self.agent.model(self.experience['state'].to(default_device()))
        self.agent.raw_action_shape=raw_action.shape
        self.agent.action=torch.argmax(raw_action,dim=1).reshape(-1,1)
        
class DiscreteEpsilonRandomSelect(AgentCallback):
    
    def __init__(self,epsilon=0.5,idx=0,min_epsilon=0.2,max_epsilon=1,max_steps=5000):
        store_attr()
    
    def before_noise(self): 
        self.mask=torch.randn(size=(self.agent.action.shape[0],))<self.epsilon
        self.experience['randomly_selected']=self.mask.reshape(-1,1)
        self.experience['epsilon']=torch.full(self.agent.action.shape,self.epsilon)
        self.experience['orignal_actions']=self.agent.action.detach().clone()
        self.agent.action[self.mask]=self.agent.action[self.mask].random_(0,self.agent.raw_action_shape[1])
        self.agent.action=self.agent.action.detach().cpu().numpy()
    
        if self.agent.model.training: 
            self.idx+=1
            self.epsilon=max(self.min_epsilon,self.max_epsilon-self.idx/self.max_steps)

In [8]:
agent=Agent(dqn,cbs=[ArgMaxFeed,DiscreteEpsilonRandomSelect(max_epsilon=0.5)]).to(default_device())
agent.do_action(state=torch.randn((10,4)).to(default_device()))


(array([[1],
        [0],
        [1],
        [0],
        [0],
        [1],
        [0],
        [0],
        [1],
        [1]]),
 {'state': tensor([[-8.5314e-01,  7.0573e-02,  1.6412e+00,  6.1500e-01],
          [-1.7369e+00,  8.7989e-01, -1.4123e+00, -8.5161e-02],
          [ 4.5644e-01,  4.4676e-01,  5.1106e-01, -8.2421e-01],
          [-5.5491e-01, -7.9168e-01, -9.0739e-01,  1.5766e+00],
          [-7.8287e-01, -5.5675e-01,  1.4355e+00,  3.4510e-01],
          [ 9.9981e-01, -9.2839e-01, -1.0311e+00,  1.4512e-03],
          [-7.8919e-01, -3.0862e+00, -7.1299e-03,  2.7443e-01],
          [ 9.8811e-01,  1.6927e+00, -1.1752e+00,  9.4141e-01],
          [-1.3542e+00,  3.1018e-01, -1.3732e+00, -6.2999e-01],
          [-2.6339e+00,  8.4253e-01,  1.4088e+00, -1.6956e+00]], device='cuda:0'),
  'randomly_selected': tensor([[ True],
          [ True],
          [False],
          [ True],
          [ True],
          [ True],
          [ True],
          [False],
          [False],
        

In [9]:
agent.show_loop()

Start Action
   - before_preprocess: []
   - after_preprocess: []
   - before_action  : [ArgMaxFeed]
   - after_action   : []
   - before_noise   : [DiscreteEpsilonRandomSelect]
   - after_noise    : []
End Action


In [10]:
# export        
class Epsilon(Metric):
    order=30
    epsilon=0

    @property
    def value(self): return self.epsilon
    def reset(self): self.epsilon=0
    def accumulate(self,learn):
        for cb in learn.model.cbs:
            if type(cb)==DiscreteEpsilonRandomSelect:
                self.epsilon=cb.epsilon

In [11]:
# export
class DQNTrainer(Callback):
    "Performs traditional training on `next_q`. Requires a callback such as `RegularNextQ`"
    def __init__(self,discount=0.99,n_steps=1):
        store_attr()
        self._xb=None  
    
    def after_pred(self): 
        self.learn.yb=self.xb
        # self.learn.xb=self.xb
        self._xb=(self.xb,)
        self.learn.done_mask=self.xb['done'].reshape(-1,)
        self.learn.next_q=self.learn.model.model(self.xb['next_state']).max(dim=1).values.reshape(-1,1)
        self.learn.next_q[self.done_mask]=0 #xb[done_mask]['reward']
        self.learn.targets=self.xb['reward']+self.learn.next_q*(self.discount**self.n_steps)
        self.learn.pred=self.learn.model.model(self.xb['state'])
        t_q=self.pred.clone()
        t_q.scatter_(1,self.xb['action'],self.targets)
        # finalize the xb and yb
        self.learn.yb=(t_q,)
        with torch.no_grad():
            self.learn.td_error=(self.pred-self.yb[0]).mean(dim=1).reshape(-1,1)**2
        
    def before_backward(self): 
        self.learn.xb=self._xb

In [12]:
# hide
SHOW_TENSOR_BOARD=True
if not os.environ.get("IN_TEST", None) and SHOW_TENSOR_BOARD:
    run_tensorboard(samples_per_plugin='images=2000')

In [48]:
# slow
dqn=DQN(4,2)
agent=Agent(dqn,cbs=[ArgMaxFeed,DiscreteEpsilonRandomSelect])
source=Source(cbs=[GymLoop('CartPole-v1',agent,steps_count=3,seed=None,
                           steps_delta=1,mode='rgb_array'),
                   FirstLast,
                   ResReduce(reduce_by=4)])

dls=SourceDataBlock().dataloaders([source],n=500,bs=1,num_workers=0)

exp_replay=ExperienceReplayCallback(bs=128,max_sz=256,warmup_sz=128,freeze_at_max=True)
er_tb=ExperienceReplayTensorboard(every_epoch=3)

learn=Learner(dls,agent,loss_func=MSELoss(),
              cbs=[exp_replay,DQNTrainer(n_steps=3),er_tb],
              metrics=[Reward,Epsilon,NEpisodes])

Could not do one pass in your dataloader, there is something wrong in it


In [49]:
slow=False
learn.fit(50 if slow else 1,lr=0.01,wd=0)

setting_ex re[lay <class '__main__.ExperienceReplayCallback'>


In [ ]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbverbose.cli import *
    make_readme()
    notebook2script()
    notebook2html()